# Evaluation des Neuronalen Netzes zur Giraffenpuzzle-Erkennung
Das Notebook [Giraffenpuzzle-Training](https://www.kaggle.com/code/toedtlifhsg/giraffenpuzzle-training?scriptVersionId=187781292) produzierte als Ausgabe ein fertig trainiertes neuronales Netz, sowie eine Aufsplittung des Giraffenpuzzleteile-Datensatzes in einen Trainings- und einen Testdatensatz. Hier wollen wir das Modell laden und auf dem Testdatensatz evaluieren.

In [ ]:
import matplotlib.pyplot as plt #Plottingroutinen zur Visualisierung
import numpy as np # Arrays- "The fundamental package for scientific computing with Python" 
import pandas as pd #Umgang mit Tabellen- hier nur zum Laden der Klassenlabels
import tensorflow as tf #Tensorflow, trainieren von neuronale Netzen (inkl. Keras)
import shutil # Dateisystemmanipulationen, insbesondere für das Kopieren von Dateien
from pathlib import Path #Pathlib, Umgang mit Dateipfaden
import scikitplot # Visualisierungen von Klassifikationsperformanz

In [ ]:
Baumstruktur_schon_vorhanden=True

In [ ]:
# Modell laden:
loaded_model = tf.keras.models.load_model('working/model/model_Geschirr.keras')
# Excel-Datei mit Angabe, ob ein Bild ein Test- oder ein Trainingsbild ist:

In [ ]:
Baumstruktur_schon_vorhanden

In [ ]:
if Baumstruktur_schon_vorhanden:
    testBaumstrukturpfad = Path('./output/geschirr/1_object_extraction_output/2_Baumstruktur_test') #MUSS ANGEPASST WERDEN
    np.set_printoptions(suppress=True, linewidth=500)
    testBilder_fn = pd.Series([fn for fn in testBaumstrukturpfad.glob('*/*')])
    testKlassenlabel = pd.Series([fn.parent.name for fn in testBilder_fn],index=[fn.name for fn in testBilder_fn])
    y_test = testKlassenlabel
else:
    train_or_test = pd.read_excel('/kaggle/input/giraffenpuzzle_beispielmodell/keras/baseline/1/train_test_split.xlsx') #MUSS ANGEPASST WERDEN
    train_or_test.columns = ['Dateiname','TrainOrTest']
    
    testBilder_fn = train_or_test[train_or_test['TrainOrTest']=='test' ]

    # Extrahiere Klassenlabel aus Dateiname
    testKlassenlabel = train_or_test.Dateiname.map(lambda s:s[:s.find('_')])
    train_or_test['Klasse'] = testKlassenlabel
    testBilder_fn = pd.Series(train_or_test[train_or_test['TrainOrTest']=='test' ])

    # Wähle nur die Testdaten
    y_test = testKlassenlabel[ train_or_test['TrainOrTest']=='test' ]

    testBilder_fn=[]
    # Kopiere die Testbilder in eine Baumstruktur
    for irow,row in testBilder_fn.iterrows():
        Dateiname,TrainOrTest,Klasse = (row.Dateiname,row.TrainOrTest,row.Klasse)
        fullfilename = Path('/kaggle/input/giraffenpuzzleteile')/Dateiname
        testBaumstrukturpfad = Path('/kaggle/working')/'test'
        (testBaumstrukturpfad/Klasse).mkdir(exist_ok=True,parents=True)
        shutil.copyfile(fullfilename,testBaumstrukturpfad/Klasse/Dateiname)
        testBilder_fn.append(testBaumstrukturpfad/Klasse/Dateiname)
    testBilder_fn = pd.Series(testBilder_fn)
y_test = pd.Series(y_test.values,index=testBilder_fn)

In [ ]:
y_test

Überprüfen wir das Resultat:

In [ ]:
!ls {testBaumstrukturpfad}
#!ls /kaggle/working/test/*   #Ausgabe ist etwas lang, aber nützlich für die Überprüfung der Vollständigkeit
print('-'*70) #nur zur visuellen Strukturierung; 3*"a" ergibt "aaa"
!ls {testBaumstrukturpfad}/loeffel-01

test_ds ist eine `tf.Dataset` ist eine Instanz einer Tensorflowklasse, welche effizient Trainingsbilder aus der Baumstruktur lädt. Natürlich ist das eigentlich erst bei grossen Datenmengen relevant, aber wir zeigen hier den Code. Wir könnten auch wieder `tf.keras.utils.image_dataset_from_directory` benutzen, aber dann würden wir den zugehörigen Dateipfad nicht erhalten. Nicht schlimm, aber wir könnten die Bilder nicht nochmals mit ihrem Label und Dateipfad zusammen anzeigen.

In [ ]:
def lade_testdaten_mit_pfad(baumstrukturpfad, image_size=(64, 64), batch_size=32, suffix='.jpg',shuffle=False):
    """
    Diese Funktion baut ein tf.Dataset basierend auf den Daten in der Baumstruktur baumstrukturpfad, welches effizient Bilddaten in das neuronale Netz füttert.
 
    """
    # Alle Bildpfade erfassen
    all_files = sorted(list(Path(baumstrukturpfad).rglob("*/*"+suffix)))  # ggf. .png anpassen
    class_names = sorted({p.parent.name for p in all_files})
    label_map = {name: idx for idx, name in enumerate(class_names)}

    # Pfade und Labels vorbereiten
    file_paths = np.array([str(p) for p in all_files])
    labels = np.array([label_map[Path(p).parent.name] for p in file_paths])
    
    def lade_bild(pfad, label):
        image = tf.io.read_file(pfad)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, image_size)
        return image, label, pfad
        
    if shuffle:
        # Gleicher Shuffle-Index für beide
        shuffle_idx = np.random.permutation(len(file_paths))
        # Koordiniert shuffeln
        file_paths_shuffled = file_paths[shuffle_idx]
        labels_shuffled = labels[shuffle_idx]
        # Dataset bauen
        ds_fn = tf.data.Dataset.from_tensor_slices((file_paths_shuffled, labels_shuffled))
        ds = ds_fn.map(lade_bild).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return ds,file_paths_shuffled, labels_shuffled, class_names
    else:
        # Dataset bauen
        ds_fn = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        ds = ds_fn.map(lade_bild).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return ds, file_paths, labels, class_names

    
test_ds, test_paths, y_test, class_names = lade_testdaten_mit_pfad(testBaumstrukturpfad, image_size=(128,128), batch_size=32, suffix='.jpg',shuffle=False)
test_ds,test_paths.shape,y_test.shape

Wir können uns ein paar Bilder, ihre Labels und die zugehörigen Dateinamen anschauen:

In [ ]:
i=np.random.randint(test_paths.shape[0])
bild = plt.imread(test_paths[i])
label = class_names[y_test[i]]
plt.imshow(bild)
plt.title(f"{label} — {Path(test_paths[i]).name}")

In [ ]:
yhat_test = loaded_model.predict(test_ds)
yhatnum_test = yhat_test.argmax(axis=1)

yhat_test.shape,y_test.shape


In [ ]:
y_test,yhatnum_test

Das ist nun also der Klassenindex- da wir die Bilder nicht "geshuffled" haben, ist zumindest die Sequenz `y_test` monoton. Bei `yhatnum_test` (also unseren Vorhersagen auf diesen Bildern) gibt es einige wenige Abweichungen. Das ist zu erwarten. Wenn wir nur wüssten, welcher Klasse das zugehört?!  
Aber weil wir zu neugierig sind, wie gut das Modell sein könnte, rechnen wir ohne dieses Wissen mal die Genauigkeit aus:

In [ ]:
Genauigkeit = np.mean(np.where(y_test==yhatnum_test,1,0)) 
# np.where könnte man auch weglassen, so: Genauigkeit = np.mean(y_test==yhatnum_test). 
# Aber was ist der Mittelwert von True, False, True? 
# Klarer ist's wenn wir mit where ersetzen: True->1 und False->0
assert Genauigkeit==np.mean(y_test==yhatnum_test) # Ergibt einen Fehler, sollte das mal nicht mehr stimmen
print(f'Die Testgenauigkeit ist {100*Genauigkeit:3.1f}%.')

Nun würden wir also auch gerne die Label kennen. Wir müssen den Labelindex (die Zahl) mit dem Namen der Klasse in Verbindung bringen. Zum Glück werden die Klassenlabel sortiert durchgezählt, so dass dieser Schritt nicht schwer ist:

In [ ]:
sorted_class_labels = np.array(sorted(testKlassenlabel.unique())) #Labels in Keras der Funktion image_dataset_from_directory werden immer alphanummerisch sortiert den Indices zugeordnet
sorted_class_labels

In [ ]:
testBilder_fn
yhatnum_test

In [ ]:
yhat_label = pd.Series(sorted_class_labels[yhatnum_test],index=test_paths)
y_test_label = pd.Series(sorted_class_labels[y_test],index=test_paths)
# das vorhergesagte Label:
yhat_label

In [ ]:
# und das Tatsächliche:
y_test_label

In [ ]:
# Natürlich muss die gleiche Genauigkeit rauskommen, wenn wir sie mit den Labels berechnen, anstatt mit den Labelindices:
Genauigkeit2 = (yhat_label==y_test_label).mean()
print(f'Die Testgenauigkeit ist {100*Genauigkeit2:3.1f}%.')

Schliesslich können wir uns noch die Wahrscheinlichkeit merken, mit welcher das Neuronale Netz eine Klasse identifiziert hat. Dies ist also die grösste Zahl unter den $P(y=c_i|X)$ für alle Klassen $c_1$,...,$c_C$ ($C$ der Anzahl Klassen unseres Problems, `yhat_test.shape[1]`.

In [ ]:
P = np.max(yhat_test,axis=1)
P.shape

# Scikit-Plot
Wir sind bereit! Wir kennen die Testbilder, deren zugehörige Klassenlabel und Vorhersagen durch das geladene Modell. Wir folgen der Dokumentation [hier](https://scikit-plot.readthedocs.io/en/stable/metrics.html). Berechnen wir Metriken! Wir beginnen mit der Confusion Matrix:

In [ ]:
from scikitplot.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_test_label,yhat_label,zero_division=0))
plot_confusion_matrix(y_test_label,yhat_label,labels=sorted_class_labels,x_tick_rotation=45);

Beachten Sie, wie hübsch blockdiagonal die confusion matrix geworden ist. Dies zeigt, dass das Modell sich meist in der Form, und selten bis nie in der Farbe irrt.  
Ein offensichtliches Problem ist natürlich, dass wir den Grossteil der Daten zum Training benutzt haben (das ist gängige Praxis), und nun nur wenige Beispiele (pro Klasse) übrig haben, um die Präzision und den Recall der Klasse zu schätzen. Mehr Daten wären schon praktisch...

In [ ]:
display((y_test_label==yhat_label).value_counts())
print('-'*30)
for curr_class_str in yhat_label.unique():
    curr_class_results=(y_test_label[y_test_label==curr_class_str]==yhat_label[y_test_label==curr_class_str])
    print(f'{curr_class_str:<10}:{100*curr_class_results.mean():3.0f}%')

In [ ]:
ax = plt.subplot(1,1,1)
scikitplot.metrics.plot_precision_recall(y_test_label,yhat_test,ax=ax)
ax.legend(bbox_to_anchor=(1.1, 1.05)); #Legende ausserhalb des Plots- sonst sieht man nichts

In [ ]:
ax = plt.subplot(1,1,1)
scikitplot.metrics.plot_roc(y_test_label,yhat_test,ax=ax)
ax.legend(bbox_to_anchor=(1.1, 1.05)); #Legende ausserhalb des Plots- sonst sieht man nichts

Ein Problem hier ist, dass das neuronale Netz für recht viele Klassen ("area=1.000") eine extreme Precision-Recall-Kurve ausgibt- von (0,1) über (1,1) nach (1,0). Aber wer würde sich schon darüber beklagen!

Zur Erinnerung, dies ist das Modell, das die obigen Resultate erziehlt hat. Wir können es grundsätzlich irgendwo (auf einem Raspberry-Pi?) laden und dort Vorhersagen generieren.

In [ ]:
loaded_model.layers[-1]

In [ ]:
loaded_model.summary()

# Conv Layer Outputs
In den nächsten paar Zeilen werden wir ("spasseshalber", ist nicht weiter wichtig), den Output der letzten Conv-Schicht ("conv2d_5") ausgeben und visualisieren.

In [ ]:
conv_layer = loaded_model.get_layer("conv2d_3")

In [ ]:
from tensorflow.keras import Model, Input

feature_model = Model(
    inputs=loaded_model.input,
    outputs=conv_layer.output,
    name="FeatureExtractor"
)


In [ ]:
from PIL import Image
import numpy as np

color_image_size = (64, 64)

# Bild laden und skalieren
im = Image.open('./output/geschirr/1_object_extraction_output/Ausschnitte/loeffel-01_0_9519.jpg')
im_resized = im.resize(color_image_size, Image.Resampling.LANCZOS)
img = np.array(im_resized) / 255.0  # Normalisierung

# Batch-Dimension hinzufügen
img_batch = img.reshape(1, *color_image_size, 3)


In [ ]:
feature_maps = feature_model.predict(img_batch)
print("Feature-Map Shape:", feature_maps.shape) 

In [ ]:
import matplotlib.pyplot as plt

# Einzeln anzeigen (z. B. die ersten 9 Kanäle)
fig, axes = plt.subplots(3, 3, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(feature_maps[0, :, :, i], cmap='viridis')
    ax.set_title(f'Filter {i}')
    ax.axis('off')
plt.tight_layout()
plt.show()
